# (11.0) Описание решаемой задачи

Необходимо проанализировать имеющиеся данные на наличие очевидных айсберг-заявок. Очевидными айсберг-заявками называются заявки, исполнение которых было произведено на сумму большую, чем было указано в выставленной заявке

## (11.1) Установка всех необходимых пакетов

In [1]:
import pymysql.cursors
import os
import mysql.connector
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, date, time

## (11.2) Определение функций

In [2]:
def ReverseToDateTime(t):
    format = '%Y%m%d%H%M%S%f'
    time_str = dt.datetime.strptime(t,format)
    return time_str

def iceberg(data_from_postgres):
    data = data_from_postgres
    #place - датафрейм (ДФ), который содержит наблюдения только по выставленным заявкам (ACTION = 1)
    #в ДФ содержатся данные по номеру заявки и выставленному объему (видимому)
    #iceberg = 1, если обнаружено, что выставленная заявка - айсберг. 0 - иначе
    #TIME_IDENT - время вскрытия заявки типа "айсберг". время указано для заявки, вскрывшей айсберг (снятие заявки или сделка) 
    place = data[data['ACTION']==1]
    place = place[['ORDERNO', 'VOLUME','SECCODE']]
    place['ICEBERG'] = 0
    place['TIME_IDENT'] = 0
    place['VOLUME_INI'] = 0
    place['VOLUME_INI'] = place['VOLUME']
    place.index = place['ORDERNO']
    
    #iceberg - ДФ с данными по снятым заявкам и по сделкам (ACTION = 0 или ACTION = 2)
    iceberg = data[data['ACTION']!=1]
    iceberg = iceberg.reset_index()
    iceberg = iceberg[['ORDERNO', 'VOLUME', 'TIME']]
    
    #Идея: 
    #ДФ place содержит только уникальные наблюдения по каждому ORDERNO (ORDERNO - уникальный ключ для ДФ place)
    #для каждой строки ДФ iceberg находит соответсвующее значение ORDERNO в ДФ place
    #если колонка VOLUME в ДФ place больше нуля, но меньше объема снятия/сделки, то код вычитает объем снятой заявки/сделки.
    #при этом в колонке TIME_IDENT выставляется время снятия/сделки (так как это действие вскрыло айсберг)
    #в колонке iceberg ставится значение 1, что идентифицирует айсберг

    #если колонка VOLUME в ДФ place больше нуля и объема снятия/сделки, код вычитает объем снятой заявки/сделки.
    #это действие необходимо, чтобы правильно идентифицировать айсберг, если он будет вскрыт в дальнейшем
    #других действий не совершается

    #в остальных случаях (если айсберг не будет вскрыт / уже был вскрыт) никаких действий с таблицей place не совершается
    #P.S. в остальных случаях на самом деле прибавляется ноль к значению VOLUME в ДФ place, но это лишь технический момент
    #прибавление нуля не повлияет на результаты
    
    for i in range(len(iceberg)):

        order_number = iceberg.loc[i, 'ORDERNO']

        if (place.loc[order_number, 'VOLUME'] >= 0) and (iceberg.loc[i, 'VOLUME'] > place.loc[order_number, 'VOLUME']):
            place.loc[order_number, 'VOLUME'] -= iceberg.loc[i, 'VOLUME']
            place.loc[order_number, 'TIME_IDENT'] = iceberg.loc[i, 'TIME']
            place.loc[order_number, 'ICEBERG'] = 1

        elif (place.loc[order_number, 'VOLUME'] >= 0) and (iceberg.loc[i, 'VOLUME'] < place.loc[order_number, 'VOLUME']):
            place.loc[order_number, 'VOLUME'] -= iceberg.loc[i, 'VOLUME']

        else:
            place.loc[order_number, 'VOLUME'] += 0
            
    #ice_detected - ДФ, содержащий все вскрытые заявки типа "айсберг" за выбранный период времени
    #Примечание: значение VOLUME отражает объем, на котором айсберг был вскрыт. 
    #В ходе дальнейших торгов VOLUME мог и дальше уходить в отрицательную зону
    ice_detected = place[place['ICEBERG']==1]
    ice_detected.index.names = ['Index']
    
    #количество вскрытых айсбергов за выбранный период времени
    #ice_numb = ice_detected['ICEBERG'].sum()
    
    #добавление времени выставления заявок, содержащих айсберги
    place = data[data['ACTION']==1]
    place_chd = place[['ORDERNO', 'BUYSELL','PRICE', 'TIME', 'DATE']]
    output = pd.merge(ice_detected, place_chd, on = 'ORDERNO', how = 'left')
    
    return output

## (11.3) Определение входных параметров 

In [12]:
# создаем список из всех названий файлов с данными (из них в дальнейшем можно извлечь дату)
orderlogs = ['OrderLog20190304.txt',
             'OrderLog20190305.txt',
             'OrderLog20190306.txt',
             'OrderLog20190307.txt',
             'OrderLog20190311.txt',
             'OrderLog20190312.txt',
             'OrderLog20190313.txt',
             'OrderLog20190314.txt',
             'OrderLog20190315.txt',
             'OrderLog20190318.txt',
             'OrderLog20190319.txt',
             'OrderLog20190320.txt',
             'OrderLog20190321.txt',
             'OrderLog20190322.txt',
             'OrderLog20190325.txt',
             'OrderLog20190326.txt',
             'OrderLog20190327.txt',
             'OrderLog20190328.txt',
             'OrderLog20190329.txt'
            ]

In [13]:
# создаем список из инструментов
tickers = ['GAZP',
           'SBER',
           'GMKN',
           'LKOH',
           'MTSS',
           'MGNT',
           'TATN',
           'NVTK',
           'YNDX',
           'ROSN',
           'FIVE',
           'VTBR',
           'SNGS',
           'CHMF',
           'ALRS'
          ]

# (11.4) Выявление айсберг-заявоки их запись в файл

In [11]:
# поочередно читаем данные по каждому инструменту из каждого файла и сохраняем данные в отдельных файлах
for l in orderlogs:
    orderlog = pd.read_csv(l,header=0)
    d = l[8:16]
    for m in tickers:
        ticker = m
        instr_orderlog = orderlog[orderlog['SECCODE']==ticker]
        c
        output = iceberg(instr_orderlog)
        output_name = 'output/' + ticker + d + '.csv' # записываем результаты в разрезе дата-инструмент в файл
        output.to_csv(output_name,index=False)

/Users/ekaterina/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# (11.5) Сбор результатов в один файл

In [15]:
# сливаем результаты из всех файлов в один датафрейм
iceberg_all = pd.DataFrame()
for l in orderlogs:
    d = l[8:16]
    for m in tickers:
        ticker = m
        output_name = 'output/' +ticker + d + '.csv'
        df_temp = pd.read_csv(output_name,header=0)
        iceberg_all = pd.concat([iceberg_all,df_temp])
iceberg_all = iceberg_all.reset_index()         
iceberg_all

/Users/ekaterina/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,index,BUYSELL,DATE,ICEBERG,ORDERNO,PRICE,SECCODE,TIME,TIME_IDENT,VOLUME,VOLUME_INI
0,0,B,20190304,1,199077,156.28,GAZP,100643595004,100936467866,-2830,2000
1,1,B,20190304,1,359638,156.18,GAZP,103027072289,105331759856,-210,2000
2,2,S,20190304,1,397852,156.58,GAZP,103657193855,103805393784,-3000,2000
3,3,B,20190304,1,486370,156.18,GAZP,105331809856,105342904115,-1000,1790
4,4,B,20190304,1,516105,156.03,GAZP,110003906212,110005256841,-1000,2000
...,...,...,...,...,...,...,...,...,...,...,...
54501,436,B,20190329,1,3821455,92.47,ALRS,183219963550,183219963586,-360,930
54502,437,B,20190329,1,3845898,92.58,ALRS,183514255473,183514256529,-1000,1000
54503,438,B,20190329,1,3845899,92.58,ALRS,183514255473,183514256529,-490,1000
54504,439,B,20190329,1,3845904,92.58,ALRS,183514257134,183514257134,-770,510


In [60]:
# добавляем расчет времени между выставлением и выявлением айсберга
iceberg_all['together_time'] = iceberg_all['DATE'].apply(str)+iceberg_all['TIME'].apply(str)
iceberg_all['together_time_identified'] = iceberg_all['DATE'].apply(str)+iceberg_all['TIME_IDENT'].apply(str)
iceberg_all['TIME'] = [ReverseToDateTime(iceberg_all.together_time[i]) for i in range (len(iceberg_all.together_time))]
iceberg_all['TIME_IDENT'] = [ReverseToDateTime(iceberg_all.together_time_identified[i]) for i in range (len(iceberg_all.together_time_identified))]
iceberg_all['DELTA'] = iceberg_all['TIME_IDENT'] - iceberg_all['TIME']
del iceberg_all['together_time']
del iceberg_all['together_time_identified']
iceberg_all

ValueError: unconverted data remains: -03-04 10:06:43.595004

In [110]:
# настраиваем типы данных
iceberg_all['VOLUME'] = iceberg_all['VOLUME'].astype('int')
iceberg_all['VOLUME_INI'] = iceberg_all['VOLUME_INI'].astype('int')
iceberg_all['DATE'] = iceberg_all['DATE'].astype('str')
iceberg_all.dtypes

index                                 int64
BUYSELL                              object
DATE                                 object
ICEBERG                              object
ORDERNO                              object
PRICE                               float64
SECCODE                              object
TIME                         datetime64[ns]
TIME_IDENT                   datetime64[ns]
VOLUME                                int64
VOLUME_INI                            int64
DELTA                       timedelta64[ns]
together_time                        object
together_time_identified             object
dtype: object

In [28]:
# сохраняем значения в файл
iceberg_all.to_csv('iceberg.csv',index=False)

# (11.6) Расчет статистики

In [24]:
# собираем статистику по всем инструментам по количеству рыночных и лимитных заявок
statistics_all = pd.DataFrame(columns = ['DATE','SECCODE','MARKET_COUNT','MARKER_VOL','LIMIT_COUNT','LIMIT_VOL','ICEBERG_COUNT','ICEBERG_VOL'])
for l in orderlogs:
    d = l[8:16]
    orderlog = pd.read_csv(l,header=0)
    for m in tickers:
        ticker = m
        market_count = len(orderlog.loc[(orderlog['PRICE']==0) & (orderlog['SECCODE']==ticker)])
        market_vol = orderlog.loc[(orderlog['PRICE']==0) & (orderlog['SECCODE']==ticker)]['VOLUME'].sum()
        limit_count = len(orderlog.loc[(orderlog['PRICE']!=0) & (orderlog['SECCODE']==ticker)])
        limit_vol = orderlog.loc[(orderlog['PRICE']!=0) & (orderlog['SECCODE']==ticker)]['VOLUME'].sum()
        iceberglog_name = 'output/' + ticker + d + '.csv'
        iceberglog = pd.read_csv(iceberglog_name,header=0)
        iceberg_count = len(iceberglog)
        iceberg_vol_ini = iceberglog.loc[(iceberglog['SECCODE']==ticker)]['VOLUME_INI'].sum()
        iceberg_vol = iceberglog.loc[(iceberglog['SECCODE']==ticker)]['VOLUME_INI'].sum() - iceberglog.loc[(iceberglog['SECCODE']==ticker)]['VOLUME'].sum()
        limit_count = limit_count - iceberg_count
        limit_vol = limit_vol - iceberg_vol_ini
        statistics_all.loc[len(statistics_all)] = [d,ticker,market_count,market_vol,limit_count,limit_vol,iceberg_count,iceberg_vol]

In [37]:
statistics_all

,DATE,SECCODE,MARKET_COUNT,MARKER_VOL,LIMIT_COUNT,LIMIT_VOL,ICEBERG_COUNT,ICEBERG_VOL
0,20190304,GAZP,5175,3196900,341398,939395640,174,540930
1,20190304,SBER,11329,8168960,421200,725092310,820,3398990
2,20190304,GMKN,4706,45796,169269,3389031,119,9053
3,20190304,LKOH,2406,50322,243075,17760148,286,75786
4,20190304,MTSS,963,148040,69730,58714310,27,169140
...,...,...,...,...,...,...,...,...
280,20190329,FIVE,676,15730,150611,42649904,28,13011
281,20190329,VTBR,3180,3011020000,172861,426002890000,276,2603310000
282,20190329,SNGS,1263,4369000,133490,1472837100,15,335200
283,20190329,CHMF,1132,81320,116562,25345670,17,32410


In [100]:
statistics_all['AVG_MARKET_VOL'] = statistics_all['MARKER_VOL']/statistics_all['MARKET_COUNT'] 
statistics_all['AVG_LIMIT_VOL'] = statistics_all['LIMIT_VOL']/statistics_all['LIMIT_COUNT'] 
statistics_all['AVG_ICEBERG_VOL'] = statistics_all['ICEBERG_VOL']/statistics_all['ICEBERG_COUNT']
statistics_all

,DATE,SECCODE,MARKET_COUNT,MARKER_VOL,LIMIT_COUNT,LIMIT_VOL,ICEBERG_COUNT,ICEBERG_VOL,AVG_MARKET_VOL,AVG_LIMIT_VOL,AVG_ICEBERG_VOL
0,20190304,GAZP,5175,3196900,341398,939395640,174,540930,617.758454,2.751614e+03,3.108793e+03
1,20190304,SBER,11329,8168960,421200,725092310,820,3398990,721.066290,1.721492e+03,4.145110e+03
2,20190304,GMKN,4706,45796,169269,3389031,119,9053,9.731407,2.002157e+01,7.607563e+01
3,20190304,LKOH,2406,50322,243075,17760148,286,75786,20.915212,7.306448e+01,2.649860e+02
4,20190304,MTSS,963,148040,69730,58714310,27,169140,153.727934,8.420237e+02,6.264444e+03
...,...,...,...,...,...,...,...,...,...,...,...
280,20190329,FIVE,676,15730,150611,42649904,28,13011,23.269231,2.831792e+02,4.646786e+02
281,20190329,VTBR,3180,3011020000,172861,426002890000,276,2603310000,946861.635220,2.464425e+06,9.432283e+06
282,20190329,SNGS,1263,4369000,133490,1472837100,15,335200,3459.224070,1.103331e+04,2.234667e+04
283,20190329,CHMF,1132,81320,116562,25345670,17,32410,71.837456,2.174437e+02,1.906471e+03


In [27]:
# сохраняем значения в файл
statistics_all.to_csv('statistics.csv',index=False)

In [82]:
statistics_all = pd.read_csv('statistics.csv',header=0)

In [101]:
# настраиваем типы данных
statistics_all['MARKET_COUNT'] = statistics_all['MARKET_COUNT'].astype('int')
statistics_all['MARKER_VOL'] = statistics_all['MARKER_VOL'].astype('int')
statistics_all['LIMIT_COUNT'] = statistics_all['LIMIT_COUNT'].astype('int')
statistics_all['LIMIT_VOL'] = statistics_all['LIMIT_VOL'].astype('int')
statistics_all['ICEBERG_COUNT'] = statistics_all['ICEBERG_COUNT'].astype('int')
statistics_all['ICEBERG_VOL'] = statistics_all['ICEBERG_VOL'].astype('int')
statistics_all['DATE'] = statistics_all['DATE'].astype('object')
statistics_all.dtypes

DATE                object
SECCODE             object
MARKET_COUNT         int64
MARKER_VOL           int64
LIMIT_COUNT          int64
LIMIT_VOL            int64
ICEBERG_COUNT        int64
ICEBERG_VOL          int64
AVG_MARKET_VOL     float64
AVG_LIMIT_VOL      float64
AVG_ICEBERG_VOL    float64
dtype: object

In [102]:
# подсчет средних значений по собранной статистике (для вставки в текст)
statistics_mean = np.round(statistics_all.pivot_table(index = 'SECCODE', aggfunc = np.mean),1)
statistics_mean

,AVG_ICEBERG_VOL,AVG_LIMIT_VOL,AVG_MARKET_VOL,ICEBERG_COUNT,ICEBERG_VOL,LIMIT_COUNT,LIMIT_VOL,MARKER_VOL,MARKET_COUNT
SECCODE,,,,,,,,,
ALRS,4263.1,2495.4,432.0,307.9,1.330669e+06,171519.7,4.259073e+08,1.183252e+06,2836.8
CHMF,1667.8,208.6,74.8,15.9,2.754840e+04,103343.6,2.217241e+07,7.606320e+04,1006.9
FIVE,344.0,231.5,21.9,56.2,1.675090e+04,113251.7,2.674378e+07,1.360450e+04,608.8
GAZP,4534.6,2562.9,644.0,345.3,1.514132e+06,406852.7,1.047982e+09,3.348828e+06,5191.6
GMKN,140.6,27.9,9.5,400.7,6.180770e+04,267028.5,7.776945e+06,3.965770e+04,4186.5
LKOH,288.6,79.2,24.7,198.0,5.443710e+04,288237.4,2.292297e+07,6.664280e+04,2713.7
MGNT,169.6,30.0,16.2,110.2,1.835090e+04,206898.9,6.308106e+06,3.975070e+04,2502.9
MTSS,3070.9,808.3,200.3,72.3,2.425179e+05,107248.8,8.678809e+07,3.194421e+05,1631.7
NVTK,1314.0,453.3,100.9,38.1,4.759110e+04,121437.5,5.442696e+07,9.153890e+04,911.8
